In [611]:
import sys
sys.path.append("../../common")

In [612]:
from aoc import *
from dataclasses import dataclass, field, replace
import heapq
import numpy as np
from tqdm import tqdm
import time
import re

from IPython.display import clear_output

In [613]:
with open("./puzzle_inputs/17.txt") as f:
    lines = f.readlines()

In [575]:
lines = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533""".split('\n')

In [576]:
def astar(initial_state, goal_check, h, init_cost=0):
    q = [(init_cost + h(initial_state), init_cost, initial_state)]
    visited = set()
    cost = 0
    while len(q) > 0:
        _, cost, state = heapq.heappop(q)
        if is_end(state):
            break
        if state in visited:
            continue
        visited.add(state)
        for ncost, neighbor in state.neighbors:
            if neighbor in visited:
                continue
            ncost = ncost + cost
            heapq.heappush(q, (ncost+h(neighbor), ncost, neighbor))
    else:
        raise Exception("Goal not found")
    return cost

In [597]:
grid = np.array([[int(c) for c in line.strip()] for line in lines])

# goal conditions
goal = np.array(grid.shape) - 1
is_end = lambda state: np.all(state.pos == goal)

# astar(State.init(grid, (0, 0)), is_end, h)

In [598]:
dijkstra(State.init(grid, (0, 0)), is_end)

Num iterations: 1 Num visited: 0
Num iterations: 14626 Num visited: 1245
Num iterations: 28923 Num visited: 1757
Num iterations: 42497 Num visited: 2134
Num iterations: 56045 Num visited: 2373
Num iterations: 69570 Num visited: 2629
Num iterations: 83084 Num visited: 2787
Num iterations: 96829 Num visited: 2922
Num iterations: 108113 Num visited: 3144
Num iterations: 121427 Num visited: 3318
Num iterations: 134492 Num visited: 3466
Num iterations: 147942 Num visited: 3615
Num iterations: 161190 Num visited: 3771
Num iterations: 174388 Num visited: 3890
Num iterations: 187444 Num visited: 3954
Num iterations: 200514 Num visited: 4121
Num iterations: 213936 Num visited: 4257
Num iterations: 227085 Num visited: 4290
Num iterations: 240033 Num visited: 4455
Num iterations: 253186 Num visited: 4588
Num iterations: 266551 Num visited: 4629
Num iterations: 279094 Num visited: 4757
Num iterations: 292043 Num visited: 4893



KeyboardInterrupt



In [485]:
deltas = {
    'u': np.array((-1, 0)),
    'd': np.array((1, 0)),
    'l': np.array((0, -1)),
    'r': np.array((0, 1))
}
reverse_direction = {
    'u': 'd',
    'd': 'u',
    'l': 'r',
    'r': 'l'
}

In [580]:
@dataclass(frozen=True, order=True)
class State:
    grid: np.ndarray = field(repr=False, hash=False, compare=False)
    pos: np.ndarray = field(compare=False)
    direction: str = field(compare=False)
    moves: int = field(compare=False)

    @classmethod
    def init(cls, grid, pos):
        return cls(grid, np.array(pos), '', 0)

    def neighbors(self):
        for ndirection, delta in deltas.items():
            if reverse_direction[ndirection] == self.direction:
                continue
            npos = self.pos + delta
            if np.any(npos < 0) or np.any(npos >= self.grid.shape):
                continue
            nmoves = self.moves+1 if ndirection == self.direction else 1
            if nmoves > 3:
                continue
            yield (self.grid[*npos], replace(self, pos=npos, direction=ndirection, moves=nmoves))

    def __eq__(self, other):
        return np.all(self.pos == other.pos) \
            and self.direction == other.direction \
            and self.moves == other.moves

    def __hash__(self):
        return hash((tuple(self.pos), self.direction, self.moves))

In [477]:
grid = np.array([[int(c) for c in line.strip()] for line in lines])

# goal conditions
goal = np.array(grid.shape) - 1
is_end = lambda state: np.all(state.pos == goal)

# heuristic
h = lambda state: np.linalg.norm(goal - state.pos, ord=1)

astar(State.init(grid, (0, 0)), is_end, h)

102

In [603]:
@dataclass(frozen=True, order=True)
class State:
    grid: np.ndarray = field(repr=False, hash=False, compare=False)
    pos: np.ndarray = field(compare=False)
    direction: str = field(compare=False)
    moves: int = field(compare=False)

    @classmethod
    def init(cls, grid, pos):
        return cls(grid, np.array(pos), '', 0)

    def neighbors(self):
        neighbors = []
        for ndirection, delta in deltas.items():
            if 0 < self.moves < 4 and self.direction != ndirection:
                continue
            if reverse_direction[ndirection] == self.direction:
                continue
            npos = self.pos + delta
            if np.any(npos < 0) or np.any(npos >= self.grid.shape):
                continue
            nmoves = self.moves+1 if ndirection == self.direction else 1
            if nmoves > 10:
                continue
            neighbors.append((self.grid[*npos], replace(self, pos=npos, direction=ndirection, moves=nmoves)))
        return neighbors

    def __eq__(self, other):
        return np.all(self.pos == other.pos) \
            and self.direction == other.direction \
            and self.moves == other.moves

    def __hash__(self):
        return hash((tuple(self.pos), self.direction, self.moves))

In [572]:
grid = np.array([[int(c) for c in line.strip()] for line in lines])

# goal conditions
goal = np.array(grid.shape) - 1
is_end = lambda state: np.all(state.pos == goal)

# heuristic
h = lambda state: np.linalg.norm(goal - state.pos, ord=1)

astar(State.init(grid, (0, 0)), is_end, h)

809

In [614]:
@dataclass(frozen=True, order=True)
class State:
    grid: np.ndarray = field(repr=False, hash=False, compare=False)
    pos: np.ndarray = field(compare=False)
    direction: str = field(compare=False)
    moves: int = field(compare=False)

    @classmethod
    def init(cls, grid, pos):
        return cls(grid, np.array(pos), '', 0)

    def neighbors(self):
        for ndirection, delta in deltas.items():
            if reverse_direction[ndirection] == self.direction:
                continue
            npos = self.pos + delta
            if np.any(npos < 0) or np.any(npos >= self.grid.shape):
                continue
            nmoves = self.moves+1 if ndirection == self.direction else 1
            if nmoves > 3:
                continue
            yield (self.grid[*npos], replace(self, pos=npos, direction=ndirection, moves=nmoves))

    def __eq__(self, other):
        return np.all(self.pos == other.pos) \
            and self.direction == other.direction \
            and self.moves == other.moves

    def __hash__(self):
        return hash((tuple(self.pos), self.direction, self.moves))

In [615]:
def astar(initial_state, neighbors, h = lambda *_: 0, init_cost=0):
    q = [(init_cost + h(initial_state), init_cost, initial_state)]
    visited = set()
    while len(q) > 0:
        _, cost, state = heapq.heappop(q)
        if state in visited:
            continue
        yield (cost, state)
        visited.add(state)
        for ncost, neighbor in neighbors(state):
            if neighbor in visited:
                continue
            ncost = ncost + cost
            heapq.heappush(q, (ncost+h(neighbor), ncost, neighbor))
    else:
        raise Exception("Goal not found")

In [616]:
grid = np.array([[int(c) for c in line.strip()] for line in lines])

# h = lambda state: np.linalg.norm(goal - state.pos, ord=1)
goal = np.array(grid.shape) - 1
for cost, state in astar(State.init(grid, (0, 0)), State.neighbors):
    if np.all(state.pos == goal):
        print(cost)
        break

KeyboardInterrupt: 

In [ ]:
# Dijkstra
grid = np.array([[int(c) for c in line.strip()] for line in lines])

goal = np.array(grid.shape) - 1

for cost, state in astar(State.init(grid, (0, 0)), State.neighbors):
    if np.all(state.pos == goal):
        print(cost)
        break